## CHATBOT

In [3]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',-1) # ini untuk melihat full column
client = MongoClient('localhost', 27017) #ihi host
db=client.get_database('fujiatma') # ini nama database sebelumnya create dlu di mongodb atau di robomongo
collection=db.get_collection('data')# ini nama collectionnya (misal data)
#collection.find_one({ "question" : "Does it fit Nook GlowLight?"})# ini contoh query saja


#cara dump ke mongo
#mongoimport --db fujiatma --collection data --file intents.json

In [4]:
cursor=collection.find() # query untuk melihat isi collection
data = pd.DataFrame(list(collection.find())) # merubah ke data frame
#data=data[data['question']!=np.nan]
#data=data.dropna(axis=0, how='all')
data.head()

,_id,answer,answerTime,answerType,asin,intents,question,questionType,unixTime
0,5a6b45346337ffb0d53c8710,Yes this fits both the nook color and the same-shaped nook tablet,"Dec 27, 2013",Y,0594033926,NaN,Is this cover the one that fits the old nook color? Which I believe is 8x5.,yes/no,1.388131e+09
1,5a6b45346337ffb0d53c8711,No. The nook color or color tablet,"Jan 5, 2015",N,0594033926,NaN,Does it fit Nook GlowLight?,yes/no,1.420445e+09
2,5a6b45346337ffb0d53c8712,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.",2 days ago,NaN,0594033926,NaN,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,open-ended,NaN
3,5a6b45346337ffb0d53c8713,yes,17 days ago,Y,0594033926,NaN,Will this fit a Nook Color that's 5 x 8?,yes/no,NaN
4,5a6b45346337ffb0d53c8714,"No, the tab is smaller than the 'color'","Feb 10, 2015",N,0594033926,NaN,will this fit the Samsung Galaxy Tab 4 Nook 10.1,yes/no,1.423555e+09


In [85]:
data_ques_answer=data[['question','answer']][:27000]
print "banyak data:", len(data_ques_answer)
data_ques_answer.head()

banyak data: 27000


,question,answer
0,Is this cover the one that fits the old nook color? Which I believe is 8x5.,Yes this fits both the nook color and the same-shaped nook tablet
1,Does it fit Nook GlowLight?,No. The nook color or color tablet
2,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,Will this fit a Nook Color that's 5 x 8?,yes
4,will this fit the Samsung Galaxy Tab 4 Nook 10.1,"No, the tab is smaller than the 'color'"


In [59]:
# proses jika di jadikan satu
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()

def preprocess(corpus):
    #melakukan proses lowercase, stopword removal, dan punctuation removal
    print "proses lowercase"
    m=[]

    for item in range(len(corpus)):
        text=string.lower(corpus['question'].iloc[item])
        #jawab=string.lower(corpus['answer'].iloc[item])
        corpus['question'].iloc[item]=text  
        #corpus['answer'].iloc[item]=jawab

    print "proses stopwords removal"
    cachedStopWords = set(stopwords.words("english"))
    for i in range(len(corpus)):
        sent=corpus['question'].iloc[i]
        kt=" ".join([word for word in sent.split() if word not in cachedStopWords])
        corpus['question'].iloc[i]=kt
        
    print "proses punctuation"
    remove=string.punctuation

    for i in range(len(corpus)):
        sent=data_ques_answer['question'].iloc[i]
        kd=' '.join(word.strip(remove) for word in sent.split())
        corpus['question'].iloc[i]=kd
    return corpus

In [86]:
data_normal=preprocess(data_ques_answer)
print 'jumlah data:',len(data_normal)
data_normal.head()

proses lowercase
proses stopwords removal
proses punctuation
jumlah data: 27000


,question,answer
0,cover one fits old nook color believe 8x5,Yes this fits both the nook color and the same-shaped nook tablet
1,fit nook glowlight,No. The nook color or color tablet
2,would fit nook 1st edition 4.9in x 7.7in,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,fit nook color that's 5 x 8,yes
4,fit samsung galaxy tab 4 nook 10.1,"No, the tab is smaller than the 'color'"


In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

In [62]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data_normal.question)
X_train_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape



tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
data_normal.question[:5]

0    cover one fits old nook color believe 8x5
1    fit nook glowlight                       
2    would fit nook 1st edition 4.9in x 7.7in 
3    fit nook color that's 5 x 8              
4    fit samsung galaxy tab 4 nook 10.1       
Name: question, dtype: object

In [70]:
clf = LinearSVC().fit(X_train_tfidf, data_normal.answer[:100])
data_normal.answer[:5]

0    Yes this fits both the nook color and the same-shaped nook tablet                                               
1    No. The nook color or color tablet                                                                              
2    I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.
3    yes                                                                                                             
4    No, the tab is smaller than the 'color'                                                                         
Name: answer, dtype: object

In [84]:
clf.score(X_train_tfidf, data_normal.answer[:100])

0.98999999999999999

In [81]:
docs_new = ''
print 'pertanyaan:\n',docs_new
docs_new=string.lower(docs_new)
docs_new1=[]
docs_new1.append(docs_new)
docs_new1

X_new_counts = count_vect.transform(docs_new1)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
print "jawab:\n", predicted

pertanyaan:
5x8
jawab:
[u'Yes']


In [43]:
X_new_counts = count_vect.transform(docs_new1)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
predicted

array([u'fit samsung galaxy tab 4 nook 10.1'], 
      dtype='<U114')

## PROSES DETAIL

In [57]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()
#porter_stemmer.stem('watching')

print data_ques_answer[:5]
m=[]
print "proses lowercase"
for item in range(5):
    text=string.lower(data_ques_answer['question'].iloc[item])
    #jawab=string.lower(data_ques_answer['answer'].iloc[item])
    #if text is float:
    #    print text
    data_ques_answer['question'].iloc[item]=text  
    #data_ques_answer['answer'].iloc[item]=jawab
    
    
    #if type(data_ques_answer['question']) is float:
    #print text   
            # test menjorok, untuk proses cepatnya saja 
print "banyak data:", len(data_ques_answer)
data_ques_answer.head()
#punctuation
#tokenisasi


#porter_stemmer.stem(string.lower(data_ques_answer.answer[2]))



                                                                      question  \
0  Is this cover the one that fits the old nook color? Which I believe is 8x5.   
1  Does it fit Nook GlowLight?                                                   
2  Would it fit Nook 1st Edition? 4.9in x 7.7in ?                                
3  Will this fit a Nook Color that's 5 x 8?                                      
4  will this fit the Samsung Galaxy Tab 4 Nook 10.1                              

                                                                                                             answer  
0  Yes this fits both the nook color and the same-shaped nook tablet                                                 
1  No. The nook color or color tablet                                                                                
2  I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.  
3  yes                             

,question,answer
0,is this cover the one that fits the old nook color? which i believe is 8x5.,Yes this fits both the nook color and the same-shaped nook tablet
1,does it fit nook glowlight?,No. The nook color or color tablet
2,would it fit nook 1st edition? 4.9in x 7.7in ?,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,will this fit a nook color that's 5 x 8?,yes
4,will this fit the samsung galaxy tab 4 nook 10.1,"No, the tab is smaller than the 'color'"


In [6]:
print "proses stopword removal"


cachedStopWords = set(stopwords.words("english"))
#more_stopwords = {'oh', 'will', 'hey', 'yet', ...}
#STOPWORDS = STOPWORDS.union(more_stopwords)
#cachedStopWords.update(reader[0][:])

#print len(data5)                       
#sent='kenapa yg ini harus!'
#data5=data4.copy()
for i in range(len(data_ques_answer)):
    sent=data_ques_answer['question'].iloc[i]
    #new_str=" ".join("".join([" " if ch in string.punctuation else ch for ch in sent]).split())
    kt=" ".join([word for word in sent.split() if word not in cachedStopWords])
    data_ques_answer['question'].iloc[i]=kt

data_ques_answer.head()

proses stopword removal


,question,answer
0,cover one fits old nook color? believe 8x5.,is this cover the one that fits the old nook color? which i believe is 8x5.
1,fit nook glowlight?,does it fit nook glowlight?
2,would fit nook 1st edition? 4.9in x 7.7in ?,would it fit nook 1st edition? 4.9in x 7.7in ?
3,fit nook color that's 5 x 8?,will this fit a nook color that's 5 x 8?
4,fit samsung galaxy tab 4 nook 10.1,will this fit the samsung galaxy tab 4 nook 10.1


In [7]:
print "proses punctuation"
remove=string.punctuation
#remove = remove.replace("@", "") 
for i in range(len(data_ques_answer)):
    sent=data_ques_answer['question'].iloc[i]
    kd=' '.join(word.strip(remove) for word in sent.split())
    data_ques_answer['question'].iloc[i]=kd
data_ques_answer.head()

proses punctuation


,question,answer
0,cover one fits old nook color believe 8x5,is this cover the one that fits the old nook color? which i believe is 8x5.
1,fit nook glowlight,does it fit nook glowlight?
2,would fit nook 1st edition 4.9in x 7.7in,would it fit nook 1st edition? 4.9in x 7.7in ?
3,fit nook color that's 5 x 8,will this fit a nook color that's 5 x 8?
4,fit samsung galaxy tab 4 nook 10.1,will this fit the samsung galaxy tab 4 nook 10.1
